In [ ]:
import os
from osgeo import gdal
import rasterio
import geopandas as gpd
import rasterio.mask
from rasterio.windows import Window
import sys
from shapely.geometry import mapping
sys.path.append(r'')
from codigos import Generar_txt
###path de yolo dentro de computadora
os.chdir(r'')
from detect_Alberto_v4 import *
from scipy.ndimage import rotate as rotate_image
from shapely import geometry
import time
import datetime

In [ ]:
raster=r""
src=rasterio.open(raster)
alto,ancho,dim,crs,H,W,minx,maxx,miny,maxy=Parametro_raster(raster,metros=10000)
alto,ancho,dim,crs,H,W,minx,maxx,miny,maxy

In [ ]:
def ampliar_shape(shape,factor_ampliacion=2):
    """Amplifica el polygon de cada manzana con el fin de extrar imagenes sin perder informacion de la manzana"""
    # shape["geometry"]=shape["geometry"]
    shape['centroid']=shape.centroid
    geometry=[]
    clase=[]
    for i,polygon in enumerate(shape['geometry']):
        try:
            point=mapping(shape['centroid'][i]).get('coordinates')
        except:
            continue
        x=point[0]
        y=point[1]
        go=[]
        coodinates=mapping(polygon).get('coordinates')[0]
        for a in coodinates:
            x1=a[0]
            y1=a[1]
            x2=x+(x1-x)*factor_ampliacion
            y2=y+(y1-y)*factor_ampliacion
            go.append((x2,y2))
        geometry.append(Polygon(go))
#         clase.append(shape.loc[i,"clase_dete"])
    return gpd.GeoDataFrame(geometry=geometry,crs="3857")

In [ ]:
# shape.envelope

In [ ]:
def color_segmetation(img1):
    mask = cv2.GaussianBlur(img1, (7, 7), 3)
    f=np.where([mask[:,:,2]<160, img1[:,:,1]>16, img1[:,:,1]<100])
    va=[i for i,x in enumerate(np.array(f)[0,:]) if x==0]
    img1[list(f)[1][va],list(f)[2][va]]=0
    w,h,_=img1.shape
    mask=cv2.threshold(img1[:,:,0],180,255,cv2.THRESH_BINARY)[1]    
    mask = cv2.dilate(mask,(5,5))
    mask=cv2.bitwise_not(mask)
    contours,_ = cv2.findContours(mask, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_TC89_L1 )
    geometry=[]
    for i in range(len(contours)):
        area = cv2.contourArea(contours[i])
        dif=np.max([x[0] for x in contours[0]],axis=0)-np.min([x[0] for x in contours[0]],axis=0)
        area2=dif[0]*dif[1]
        if area>=.6*area2 and area>dim and area<H*W:
            geometry.append(Polygon([[map_d(x[0][0],0,W,minx,maxx),map_d(x[0][1],H,0,miny,maxy)] for x in contours[i]]))

    shp_contorno=gpd.GeoDataFrame(geometry=geometry,crs=3857)
    shp_contorno["area"]=shp_contorno.area
    shp_contorno=shp_contorno.sort_values(by="area",ascending=False).reset_index(drop=True)[1:]
    # shp_contorno["geometry"]=shp_contorno["geometry"]
    # shp_contorno["a1"]=shp_contorno["geometry"].area
    # shp_contorno["a2"]=shp_contorno["geometry"].envelope.area
    # shp_contorno=shp_contorno[shp_contorno["a1"]>=.2*shp_contorno["a2"]]
#     shp_contorno=shp_contorno[shp_contorno["area"]>1000]
#     shp_contorno.plot()
    return shp_contorno